In [127]:
from OSMPythonTools.api import Api
from OSMPythonTools.overpass import Overpass, overpassQueryBuilder
from OSMPythonTools.nominatim import Nominatim
import math
import numpy as np
import pandas as pd

In [128]:
DISTANCE = 5

In [129]:
def deg2rad(deg):
  return deg * (math.pi/180)

def getDistanceFromLatLonInKm(coords1,coords2):
    R = 6371 #Radius of the earth in km
    dLat = deg2rad(coords2[0] - coords1[0])
    dLon = deg2rad(coords2[1] - coords1[1])

    a = math.sin(dLat/2) * math.sin(dLat/2) + math.cos(deg2rad(coords1[0])) * math.cos(deg2rad(coords2[0])) * math.sin(dLon/2) * math.sin(dLon/2)
   
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a)); 
    d = R * c * 1000 # Distance in m
    
    return d

In [130]:
op = Overpass()
nm = Nominatim()

In [131]:
area = nm.query("Zurich, Switzerland")
area.areaId()

3601682248

In [132]:
fountain_query = overpassQueryBuilder(area=area.areaId(), elementType='node', selector='"amenity"="drinking_water"')
fountain_output = op.query(query=fountain_query)
fountain_output.countNodes()

944

In [133]:
benches_query = overpassQueryBuilder(area=area.areaId(), elementType='node', selector='"amenity"="bench"')
benches_output = op.query(query=benches_query)
benches_output.countNodes()

6260

In [134]:
benches_coords = [(x.lat(), x.lon()) for x in benches_output.elements()]
fountain_coords = [(x.lat(), x.lon()) for x in fountain_output.elements()]

In [135]:
cross = np.zeros((len(fountain_coords), len(benches_coords)))
for i, f in enumerate(fountain_coords):
    for j, b in enumerate(benches_coords):
        cross[i,j] = getDistanceFromLatLonInKm(f, b)
cross.shape

(944, 6260)

In [136]:
map = cross < DISTANCE

In [137]:
reduced = np.any(map, axis=1)
indeces, = np.where(reduced)

In [138]:
elems = np.array(fountain_output.elements())
final = elems[indeces]
len(final)

111

In [139]:
ids = np.array([x.id() for x in final])
ids

array([   60733338,    60734353,    61272331,    73242944,    83325695,
          83326029,    84507238,    84511951,   102713251,   240714137,
         262378043,   287423001,   333187280,   345831607,   353222181,
         366921147,   368348027,   373510745,   385443211,   448855235,
         448855247,   448855255,   450928154,   471792545,   474738688,
         481443885,   492010054,   500913369,   667406248,   672749756,
         684346495,   698059005,   767253408,   767502384,   792594100,
         853916702,   875617216,   891896177,   922778609,   943929803,
        1274907098,  1274907110,  1375539737,  1397530120,  1517967595,
        1614556178,  1735294011,  1756233049,  1819650977,  1878383146,
        1943556638,  1977731106,  2121032955,  2202847989,  2211167508,
        2214821372,  2222137103,  2222137107,  2288917833,  2340248120,
        2459524577,  2583985953,  2665685703,  2688268601,  2820206990,
        2917165571,  2930249899,  3000057373,  3441413306,  3447